In [13]:
import boto3
import json

bedrock = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')
bedrock_agent = boto3.client(service_name='bedrock-agent')

MODEL_ID = "amazon.nova-micro-v1:0"


In [14]:
temperature = .7

inference_config = {"temperature": temperature}

system_prompts = [{"text": "You are a virtual travel assistant that suggests destinations based on user preferences."
                + "Only return destination names and a brief description."}]

messages = []

message_1 = {
    "role": "user",
    "content": [{"text": "Create a list of 3 travel destinations."}]
}

messages.append(message_1)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

def print_response(response):
    model_response = response.get('output', {}).get('message', {}).get('content', [{}])[0].get('text', '')

    print("✈️ Your suggested travel destinations:")
    print(model_response)

print_response(response)

✈️ Your suggested travel destinations:
1. **Kyoto, Japan**
   - Experience traditional Japanese culture with stunning temples, gardens, and tea ceremonies.

2. **Banff National Park, Canada**
   - Enjoy breathtaking mountain scenery, outdoor activities like hiking and skiing, and unique wildlife.

3. **Santorini, Greece**
   - Discover charming white-washed villages, stunning sunsets, and crystal-clear blue waters.


In [15]:
message_2 = {
        "role": "user",
        "content": [{"text": "Only suggest travel locations that are no more than one short flight away."}]
}

messages.append(message_2)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
1. **Orlando, Florida**  
   Description: Known for its world-famous theme parks like Disney World and Universal Studios, Orlando offers a mix of family-friendly attractions, vibrant nightlife, and natural beauty.

2. **San Diego, California**  
   Description: Famous for its beautiful beaches, iconic San Diego Zoo, and bustling Seaport Village, this coastal city combines relaxation and adventure seamlessly.

3. **Miami, Florida**  
   Description: Renowned for its vibrant culture, stunning beaches, and lively nightlife, Miami offers a unique blend of Latin and Caribbean influences, perfect for a tropical getaway.


In [16]:
message_3 = {
        "role": "user",
        "content": [{"text": "List destinations by their proximity to Seattle, WA USA."}]
}

messages.append(message_3)

response = bedrock.converse(
    modelId=MODEL_ID,
    messages=messages,
    system=system_prompts,
    inferenceConfig=inference_config
)

print_response(response)

✈️ Your suggested travel destinations:
1. **Portland, Oregon**
   - A vibrant city known for its eclectic arts scene, quirky coffee shops, and beautiful waterfront.

2. **Vancouver, British Columbia**
   - A stunning city surrounded by mountains and ocean, famous for its diverse culture, Stanley Park, and vibrant food scene.

3. **Victoria, British Columbia**
   - A charming capital city with Victorian architecture, lush gardens, and the historic Butchart Gardens.


In [17]:
try:
    response = bedrock_agent.create_prompt(
        name="Travel-Agent-Prompt",
        description="Checks if all trip information has been provided.",
        variants=[
            { 
                "name": "Variant1",
                "modelId": MODEL_ID,
                "templateType": "CHAT",
                "inferenceConfiguration": {
                    "text": {
                        "temperature": 0.4
                    }
                },
                "templateConfiguration": { 
                    "chat": {
                        'system': [ 
                            {
                                "text": """You are a travel agent evaluating trip requests for custom itineraries. 
                                Review the message carefully and answer YES or NO to the following screening questions. 
                                Be strict—if any detail is missing or unclear, answer NO.

                                A) Is the destination clearly stated?
                                B) Are the travel dates within a reasonable range (not last−minute or over a year away)?
                                C) Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?
                                D) Is there any mention of a valid passport or travel documentation?
                                E) Is there enough information to follow up with a proposed itinerary?"""
                            }
                        ],
                        'messages': [{
                            'role': 'user',
                            'content': [ 
                                {
                                    'text': "Trip request: {{event_request}}"
                                }
                            ]
                        }],
                        'inputVariables' : [
                            { 'name' : 'event_request'}
                        ]
                    }
                }
        }]
    )
    print("Created!")
    prompt_arn = response.get("arn")
except bedrock.exceptions.ConflictException as e:
    print("Already exists!")
    response = bedrock.list_prompts()
    prompt = next((prompt for prompt in response['promptSummaries'] if prompt['name'] == "TripBooker_xyz"), None)
    prompt_arn = prompt['arn']

prompt_arn

Created!


'arn:aws:bedrock:us-east-1:257317331274:prompt/W2ATFDFA7C'

In [18]:
response = bedrock.converse(
    modelId=prompt_arn,
    promptVariables={
        'event_request': {
            'text': """
                Hi there! I'm planning a trip to Italy with my partner and would love some help organizing the itinerary. We're hoping to travel between September 10–20 this year, ideally flying into Rome and spending a few days in Florence and Venice as well. We’d love recommendations on tours, cultural sites, and good local restaurants. We’re not interested in anything risky like skydiving or hiking remote trails — just want a relaxing and enriching experience. We both have valid passports. Let me know what other details you need!
                """
        }
    },
)
print(response['output']['message']['content'][0]['text'])

YES

Let's break down the trip request against each screening question:

A) Is the destination clearly stated?
- Yes, the destination is clearly stated as Italy.

B) Are the travel dates within a reasonable range (not last−minute or over a year away)?
- Yes, the travel dates are within a reasonable range (September 10–20 this year).

C) Does the request avoid high−risk or restricted activities (e.g., extreme sports, off−grid travel)?
- Yes, the request specifies a preference for a relaxing and enriching experience without high-risk activities like skydiving or remote hiking trails.

D) Is there any mention of a valid passport or travel documentation?
- Yes, both individuals have valid passports.

E) Is there enough information to follow up with a proposed itinerary?
- Yes, there is enough information provided to start crafting an itinerary, including specific cities (Rome, Florence, Venice) and a general preference for cultural experiences and local restaurants.

Therefore, the answer 